In [54]:
%pip install -r requirements.txt

  Using cached AST-0.0.2.tar.gz (19 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/qf/ngth87sn0_j9_2j6hl6_jlz40000gn/T/pip-install-mp4pf830/ast_2a03d15c3a30452395348b0dcc447009/setup.py", line 6, in <module>
          README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
        File "/Users/yida/.pyenv/versions/3.10.4/lib/python3.10/codecs.py", line 905, in open
          file = builtins.open(filename, mode, buffering)
      FileNotFoundError: [Errno 2] No such file or directory: '/private/var/folders/qf/ngth87sn0_j9_2j6hl6_jlz40000gn/T/pip-install-mp4pf830/ast_2a03d15c3a30452395348b0dcc447009/AST/README'
      [end of output]
  
  note: This err

In [55]:
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.nn.functional import pairwise_distance
from torchvision.transforms.functional import InterpolationMode

# Load BLIP model

In [56]:
import sys
sys.path.append('BLIP')
from BLIP.models.blip import blip_feature_extractor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 512
blip_model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base.pth'
blip_model = blip_feature_extractor(pretrained=blip_model_url, image_size=image_size, vit='base', med_config='BLIP/configs/med_config.json')
blip_model.eval()
blip_model = blip_model.to(device)

reshape position embedding from 196 to 1024
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base.pth


In [87]:
def blip_load_image(image_input,image_size,device):

    raw_image = image_input.convert('RGB')   

    w,h = raw_image.size
#     display(raw_image.resize((w//5,h//5)))
    
    transform = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 
    image = transform(raw_image).unsqueeze(0).to(device)   
    return image

# Load Siamese model

In [88]:
class SiameseNetwork(nn.Module):
    def __init__(self, input_size, embedding_size=128):
        super(SiameseNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(256, embedding_size),
        )
    
    def forward(self, x):
        x = self.fc(x)
        return x

In [89]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights_path = '5_epoch_3linear_dropout_lr_5.pt'
siamese_model = SiameseNetwork(input_size=768, embedding_size=128).to(device)
siamese_model.load_state_dict(torch.load(weights_path, map_location=device))
siamese_model.eval()

SiameseNetwork(
  (fc): Sequential(
    (0): Linear(in_features=768, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
  )
)

# Load images

In [121]:
df_all_images = pd.read_csv('all_blip_siamese.csv')
# set image_path as index
df_all_images = df_all_images.set_index('image_path')

In [122]:
df_all_images.head()

,Unnamed: 0,group,blip,blip_siamese
image_path,,,,
sofas/sofa-beds/91503_image_0_resized.jpg,0,453,"[0.38693681359291077, 0.19435164332389832, -0....","[0.01813609153032303, -0.14332135021686554, -0..."
sofas/4-seaters-up/94024_image_0_resized.jpg,1,104,"[0.6286948919296265, -0.5614968538284302, -0.1...","[0.22950240969657898, -0.12034033983945847, -0..."
sofas/l-shape/91717_image_0_resized.jpg,2,185,"[0.42798036336898804, 0.20924703776836395, 0.0...","[0.09146980941295624, -0.23594890534877777, -0..."
sofas/4-seaters-up/88163_image_0_resized.jpg,3,115,"[0.021315090358257294, 0.19990620017051697, -0...","[0.23008421063423157, -0.1584370881319046, -0...."
chairs/office-chairs/25435_image_0_resized.jpg,4,811,"[0.2223481684923172, 0.12761016190052032, -0.0...","[0.19579604268074036, -0.28381842374801636, 0...."


In [123]:
def resize_pad_image(image):
    try:
        img = image
        scaling_factor = 512.0 / max(img.size)
        new_size = (int(img.size[0] * scaling_factor), int(img.size[1] * scaling_factor))
        img_resized = img.resize(new_size, Image.LANCZOS)
        new_img = Image.new("RGB", (512, 512), "white")
        new_img.paste(img_resized, ((512 - new_size[0]) // 2, (512 - new_size[1]) // 2))
        return new_img
    except Exception as e:
        print(f"Error processing {image}: {e}")
        return None

In [124]:
def process_input_image(input_image):
    resized_image = resize_pad_image(input_image)
    blip_image = blip_load_image(resized_image, 512, device)
    blip_embedding = blip_model(blip_image, '', mode='image')[0,0].tolist()
    siamese_embedding = siamese_model(torch.tensor(blip_embedding).float().to(device)).detach()
    return siamese_embedding

In [125]:
siamese_embeddings = torch.stack([torch.tensor(ast.literal_eval(embedding)) for embedding in df_all_images['blip_siamese']])

def predict_similar_furniture(embedding):
    siamese_distances = pairwise_distance(embedding, siamese_embeddings)
    
    _, top_5_indices_siamese = torch.topk(siamese_distances, 5, largest=False)
    top_5_image_paths_siamese = df_all_images.iloc[top_5_indices_siamese.cpu().tolist()].index.tolist()
    top_5_image_paths_siamese = [f"../training-evaluation/all/{image_path}" for image_path in top_5_image_paths_siamese]
    
    return top_5_image_paths_siamese

# Run the app

In [141]:
import gradio as gr

def recommend_similar_furniture(input_image):
    if isinstance(input_image, np.ndarray):
        input_image = Image.fromarray(input_image)
    siamese_embedding = process_input_image(input_image)
    similar_images = predict_similar_furniture(siamese_embedding)
    print(similar_images)
    return similar_images

output_images = []
with gr.Blocks() as demo:
    with gr.Row():
        input_image = gr.Image(type='pil', label='Upload Image', height=512)
    with gr.Row():
        for i in range(3):
            with gr.Column():
                text = gr.Label(f'Top {i+1}')
                output_image = gr.Image(type='pil', label=f'Top {i+1}', width=512)
                output_images.append(output_image)
    with gr.Row():
        for i in range(2):
            with gr.Column():
                text = gr.Label(f'Top {i+4}')
                output_image = gr.Image(type='pil', label=f'Top {i+4}', width=512)
                output_images.append(output_image)
        
    input_image.change(fn=recommend_similar_furniture, inputs=[input_image], outputs=output_images)

demo.launch()

Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.
